In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/navires-2024-fait-main/ships24.tgz


In [2]:
!tar xzf /kaggle/input/navires-2024-fait-main/ships24.tgz

In [3]:
!ls /kaggle/working

__notebook__.ipynb  ships_gray	test.npy


In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt
import tensorflow as tf

SEED = 123                 # to be able to rerun the same NN
np.random.seed(SEED)
tf.random.set_seed(SEED)

np.set_printoptions(precision=4, suppress=True, floatmode='fixed')

%matplotlib inline

2024-06-01 10:29:00.828489: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 10:29:00.828591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 10:29:00.967939: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

directory='/kaggle/working/ships_gray'

datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

training_dataset = datagen.flow_from_directory(
    directory,
    class_mode='categorical',
    color_mode='grayscale',
    batch_size=32,
    target_size=(128, 192),
    shuffle=True,
    seed=72,
    subset='training',
    interpolation='bilinear',
)

validation_dataset  = datagen.flow_from_directory(
    directory,
    class_mode='categorical',
    color_mode='grayscale',
    batch_size=32,
    target_size=(128, 192),
    shuffle=True,
    seed=72,
    subset='validation',
    interpolation='bilinear',
)

Found 33768 images belonging to 10 classes.
Found 8436 images belonging to 10 classes.


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.regularizers import l2

# Définir le modèle
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(128, 192, 1)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(512, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    GlobalAveragePooling2D(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(len(training_dataset.class_indices), activation='softmax')
])


# Afficher le nombre de couches
print("Nombre de couches : ", len(model.layers))

# Compiler le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Nombre de couches :  28


In [7]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

model.fit(x=training_dataset, epochs=30, validation_data=validation_dataset, callbacks=[early_stopping])

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-06-01 10:29:26.468531: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 4.34056, expected 3.36996
2024-06-01 10:29:26.468596: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 3.69911, expected 2.72851
2024-06-01 10:29:26.468605: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 3.39073, expected 2.42013
2024-06-01 10:29:26.468613: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 3.67995, expected 2.70936
2024-06-01 10:29:26.468621: E external/local_xla/xla/service/gpu/buffer_comparator.cc:11

   2/1056 ━━━━━━━━━━━━━━━━━━━━ 1:25 81ms/step - accuracy: 0.0312 - loss: 11.3781  

I0000 00:00:1717237786.440251      84 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 990/1056 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - accuracy: 0.4002 - loss: 4.6968

2024-06-01 10:31:04.644449: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 3.73495, expected 3.24058
2024-06-01 10:31:04.644517: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 3.17536, expected 2.68099
2024-06-01 10:31:04.644532: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.91418, expected 2.41981
2024-06-01 10:31:04.644551: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 3.54452, expected 3.05015
2024-06-01 10:31:04.644563: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 2.83743, expected 2.34306
2024-06-01 10:31:04.644573: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 3.29695, expected 2.80258
2024-06-01 10:31:04.644585: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.76565, expected 2.27128
2024-06-01 10:31:04.644594: E external/local_xla/xla/se

1056/1056 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - accuracy: 0.4061 - loss: 4.5711

2024-06-01 10:31:27.216840: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 101: 1.29365, expected 1.04811
2024-06-01 10:31:27.216933: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 646: 1.44418, expected 1.19864
2024-06-01 10:31:27.216951: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1013: 1.25238, expected 1.00684
2024-06-01 10:31:27.216970: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1087: 1.15504, expected 0.909496
2024-06-01 10:31:27.216984: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1270: 1.43624, expected 1.1907
2024-06-01 10:31:27.216995: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1280: 1.43212, expected 1.18658
2024-06-01 10:31:27.217007: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1369: 1.37886, expected 1.13331
2024-06-01 10:31:27.217018: E extern

1056/1056 ━━━━━━━━━━━━━━━━━━━━ 137s 100ms/step - accuracy: 0.4062 - loss: 4.5692 - val_accuracy: 0.2278 - val_loss: 2.8383
Epoch 2/30
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 84s 79ms/step - accuracy: 0.6573 - loss: 1.2205 - val_accuracy: 0.6766 - val_loss: 1.1970
Epoch 3/30
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 83s 79ms/step - accuracy: 0.7356 - loss: 1.0144 - val_accuracy: 0.6824 - val_loss: 1.2168
Epoch 4/30
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 84s 79ms/step - accuracy: 0.7956 - loss: 0.8424 - val_accuracy: 0.7467 - val_loss: 1.1016
Epoch 5/30
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 83s 79ms/step - accuracy: 0.8225 - loss: 0.7889 - val_accuracy: 0.7057 - val_loss: 1.1571
Epoch 6/30
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 83s 79ms/step - accuracy: 0.8478 - loss: 0.7072 - val_accuracy: 0.8281 - val_loss: 0.8589
Epoch 7/30
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 83s 79ms/step - accuracy: 0.8703 - loss: 0.6416 - val_accuracy: 0.8125 - val_loss: 0.7956
Epoch 8/30
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 83s 79ms/step - accuracy: 0.8807 - loss: 0.

In [8]:
X_test = np.load('/kaggle/working/test.npy', allow_pickle=True)
X_test = X_test.astype('float32') / 255

In [9]:
res = model.predict(X_test).argmax(axis=1)
df = pd.DataFrame({"Category":res})
df.to_csv("reco_nav.csv", index_label="ID")

31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

2024-06-01 11:07:35.342266: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 0: 2.81281, expected 1.81633
2024-06-01 11:07:35.342331: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 1: 3.23289, expected 2.23642
2024-06-01 11:07:35.342348: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 2: 2.68713, expected 1.69066
2024-06-01 11:07:35.342366: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 3: 2.57484, expected 1.57836
2024-06-01 11:07:35.342379: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 4: 2.91767, expected 1.9212
2024-06-01 11:07:35.342389: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 5: 2.92405, expected 1.92758
2024-06-01 11:07:35.342400: E external/local_xla/xla/service/gpu/buffer_comparator.cc:1137] Difference at 6: 2.28038, expected 1.2839
2024-06-01 11:07:35.342411: E external/local_xla/xla/serv

32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 185ms/step


In [10]:
!head reco_nav.csv

ID,Category
0,5
1,2
2,2
3,3
4,8
5,8
6,4
7,2
8,3


In [11]:
import os
os.chdir(r'/kaggle/working')
from IPython.display import FileLink
FileLink(r'reco_nav.csv')

/kaggle/working/reco_nav.csv